# 파이토치 기본 익히기

* 대부분의 workflow는 데이터셋 작업, 모델 생성, optimizer, 모델 저장으로 이루어짐. 여기선 이런 개념들에 대해 자세히 알아본다.
* FashionMNIST를 이용하여 multiclassification을 구현한다.
* https://tutorials.pytorch.kr/beginner/basics/tensorqs_tutorial.html

# Quick Start!

quick start를 통해 기본 API를 확인한다. 이후 뒷 section으로 각각을 더 자세히 알아본다.

## dataset 작업
* `torch.utils.data.Dataset`, `torch.utils.data.DataLoader`을 알아본다.
* Dataset : 샘플과 label을 저장, transform 포함
* Dataloader는 iterable 객체로 만든다.
  * Dataset을 인자로 받는다. 
  * iterable 객체로 만들며 batch, sampling, shuffle, multiprocess data loading을 지원한다.
  * (images, label) 형태의 묶음으로 batch 반환한다.

* torchvision.dataset : 
  * 비전 관련된 공개 데이터 셋을 제공
  * 공개 데이터셋 목록 링크 : https://pytorch.org/vision/stable/datasets.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets    # 비전 관련된 공개 데이터셋을 제공
from torchvision.transforms import ToTensor

In [17]:
batch_size = 64
learning_rate = 1e-3
epochs = 5

model_f = 'model.pth'
gpu_idx = 6

In [4]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

train_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# test_loader의 데이터 확인
for x, y in test_loader:
    print(x.shape, x.dtype)  # [64, 1, 28, 28], torch.float32
    print(y.shape, y.dtype)  # [64], torch.int64
    break

torch.Size([64, 1, 28, 28]) torch.float32
torch.Size([64]) torch.int64


## model 정의
* `nn.Module`을 상속받는 클래스를 만들어 정의한다.
* `__init__`에서 layer들을 정의
* `forward`에서 구조를 정의한다.
* GPU가 있다면 model 객체를 생성하여 `.cuda()` 혹은 `.device('cuda:GPUidx')`로 GPU메모리로 모델을 이동시킨다.

In [5]:

device = torch.device(f'cuda:{gpu_idx}' if torch.cuda.is_available() else 'cpu')
print(device)

class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN().to(device)
model

cuda:6


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

## parameter optimizing
loss, optimizer 정의

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## training precessing 정의
전체 for문으로 작성하지 않고 train, eval함수를 각각 정의해서 진행해본다.

In [13]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    size = len(dataloader.dataset)   # !! 전체 sample 갯수 출력 !! 첨 알았음.
    for batch_idx, data in enumerate(dataloader):
        x, y = data[0].to(device), data[1].to(device)
        
        out = model(x)
        loss = criterion(out, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            loss = loss.item()
            current = batch_idx * len(x)  # 현재까지 학습시킨 샘플 갯수
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, criterion):
    model.eval()
    test_loss, correct = 0, 0
    num_batch = len(dataloader)
    size = len(dataloader.dataset)
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader):
            x, y = data[0].to(device), data[1].to(device)
            out = model(x)
            loss = criterion(out, y)
            test_loss += loss.item()
            out_label = torch.argmax(out, dim=1)   # out(one hot)을 label로 변환
            correct += (out_label == y).sum().item()
    
    # loss는 C.E() default가 sum이 아닌 avg이므로(reduction='mean') 배치 단위 평균이 계산 되어있음. 즉, 전체 평균을 만들기 위해선 배치로 나눠야함
    # 반명 accuracy는 위의 식에서 모든 샘플에 대해서 계산되어있음. 즉, 전체 평균을 만들기 위해선 샘플로 나눠야 함
    test_loss /= num_batch
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
for epoch in range(epochs):
    print(f'now epoch {epoch+1}')
    print('--------------------------')
    
    train(train_loader, model, criterion, optimizer)
    test(test_loader, model, criterion)
print('done!!!')

now epoch 1
--------------------------
loss: 2.171953  [    0/60000]
loss: 2.160245  [ 6400/60000]
loss: 2.102440  [12800/60000]
loss: 2.134495  [19200/60000]
loss: 2.084115  [25600/60000]
loss: 2.010784  [32000/60000]
loss: 2.061188  [38400/60000]
loss: 1.967415  [44800/60000]
loss: 1.974083  [51200/60000]
loss: 1.910295  [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 1.907557 

now epoch 2
--------------------------
loss: 1.931993  [    0/60000]
loss: 1.903503  [ 6400/60000]
loss: 1.787462  [12800/60000]
loss: 1.846663  [19200/60000]
loss: 1.732368  [25600/60000]
loss: 1.665687  [32000/60000]
loss: 1.713885  [38400/60000]
loss: 1.600342  [44800/60000]
loss: 1.625012  [51200/60000]
loss: 1.523332  [57600/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.542013 

now epoch 3
--------------------------
loss: 1.605543  [    0/60000]
loss: 1.567723  [ 6400/60000]
loss: 1.419542  [12800/60000]
loss: 1.506820  [19200/60000]
loss: 1.383631  [25600/60000]
loss: 1.360498  [32000/60000]

## model save & loading
* saving `torch.save()`, `model.state.dict()`
* loading `torch.load()`, `model.load_state.dict()`

In [18]:
# 모델 저장
torch.save(model.state_dict(), model_f)
print(f'model 저장 완료 : {model_f}')

model 저장 완료 : model.pth


In [19]:
# 모델 불러오기
model = NN()
model.load_state_dict(torch.load(model_f))

<All keys matched successfully>

## 로드한 모델로 prediction.

In [35]:
# test_dataset을 이용한 방식

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
img, label = test_data[0][0], test_data[0][1]
print(img.shape)
print(label)

with torch.no_grad():
    pred = model(img)
#     print(pred.shape) # [1, 10]
    pred_label = pred[0].argmax(0)
    
    print(f'predicted label:{classes[pred_label]}')
    print(f'GT label: {classes[label]}')

torch.Size([1, 28, 28])
9
predicted label:Ankle boot
GT label: Ankle boot


In [36]:
# 다른 방식 데이터 로딩: test_loader를 이용한 방식

model.eval()
aa = iter(test_loader).next()
img, label = aa[0], aa[1]
print(img.shape)  # [64, 1, 28, 28]
print(label.shape)  # [64]

with torch.no_grad():
    pred = model(img)
    print(pred.shape) # [64, 10]
    pred_label = pred[0].argmax(0)
    
    print(f'predicted label:{classes[pred_label]}')
    print(f'GT label: {classes[label[0]]}')

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 10])
predicted label:Ankle boot
GT label: Ankle boot
